## Bibliotecas utilizadas

In [77]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from pprint import pprint
from ipyleaflet import *
from ipyleaflet import Map, Marker

## Classe e métodos 

In [106]:
class Preparador:
    
    # Classe responsável pelo carregamento inicial do dataframe
    def __init__(self, dataset):
        self.df = pd.read_csv(dataset, sep='\n', delimiter=';')
        self.fields = list(self.df)
        self.n_lines = len(self.df)

        
    def get_df(self):
        return self.df
    
    # Para cada coluna identique a quantidade de linhas com dados faltantes (em alguns casos,
    # o dado faltante é uma string vazia, em outros casos é uma string contendo algum valor do tipo:
    # "sem informação"). Faça um método que retorna a média de dados faltantes por coluna
    def search_nan(self):
        count = {}
        for field in self.fields:
            count[field] = 100*sum(
                [1 if value == "" or value == "Sem Informações" else 0 for value in self.df[field]]) / self.n_lines
        return count

    # Para cada item identifique até qual nível taxônomico a ocorrência foi identificada.
    def n_tax(self):
        count = {}
        for i in range(self.n_lines):
            count[i] = ""
            for field in reversed(['Reino', 'Filo', 'Classe', 'Ordem', 'Familia', 'Genero', 'Especie']):
                if self.df.loc[i,field] != "Sem Informações" and self.df.loc[i,field] != "":
                    count[i] = field
                    break
        return count

    # Monte filtros de ocorrências por estados, nome de espécie (nome exato ou parte do nome)
    #  e categoria de ameaça, e outros filtros que julgar relevante.
    def filter(self):
        print('Filtrar dados por:')
        print('1. Estado')
        print('2. Nome de espécie')
        print('3. Categoria de ameaça')
        n = int(input('Digite o número da opção: '))
        if n==1:
            est = str(input('Digite a sigla do estado: '))
            new_df = self.df[self.df['Estado/Provincia'] == est]
            
        elif n==2:
            esp = str(input('Digite o nome da espécie: '))
            new_df = self.df[self.df['Especie'] == esp]        
            
        elif n==3:
            ameaca = str(input('Digite a categoria de ameaça: '))
            new_df = self.df[self.df['Categoria de Ameaca'] == ameaca]
        print("Ocorrências com o filtro aplicado:", len(new_df))
        pprint(new_df)
        return new_df

    # Crie uma funcionalidade para avaliar se a informação de longitude e latitude corresponde
    #  a informação presente na localização
    def verify_lat_long(self, city, lat, long):
        key = '65b25c705a5349ad99c824ca809363b7'
        geocoder = OpenCageGeocode(key)
        results = geocoder.reverse_geocode(lat, long)
        pprint(results)
        try:
            r_city = results[0]['components']['city']
        except Exception as identifier:
            try:
                r_city = results[0]['components']['island']
            except Exception as identifier:
                r_city = ""
        self.pbar.update(1)
        return r_city == city

    # Plota coordenadas especificas no mapa
    # Aqui é possível verificar visualmente o local de ocorrência de diferentes espécies
    def plot_map(self, newdf):
        my_center = geocoder.geocode(u'Salvador, Brazil')
        center = (my_center[0]['geometry']['lat'], my_center[0]['geometry']['lng'])
        map = Map(center=center, zoom=5)
        # loc1 =(self.df["Latitude"][0], self.df["Longitude"][0])
        # loc2 = (self.df["Latitude"][250], self.df["Longitude"][250])
        # marker1 = Marker(location=loc1, draggable=True)
        # marker2 = Marker(location=loc2, draggable=True)
        # map.add_layer(marker1)
        # map.add_layer(marker2)
        for i in newdf.index:
            loc =(newdf["Latitude"][i], newdf["Longitude"][i])
            marker = Marker(location=loc)
            map.add_layer(marker)
        mymap = map
        return(mymap)
        
        
    def call(self):
        futures = {}
        count = {}
        self.pbar = tqdm(total=len(self.df))
        with ThreadPoolExecutor(max_workers=100) as executor:
            for i, (lat, long, city) in enumerate(zip(self.df['Latitude'], self.df['Longitude'], self.df['Municipio'])):
                futures[i] = executor.submit(self.verify_lat_long, city, lat, long)

        for i, f in futures.items():
            count[i] = f.result()
        return countpreparador.plot_map()

## Criação da instância da classe e utilização dos métodos

In [107]:
# Inserção da key do geocoder
key = 'fc805050230944ecabeae3cc1eeefc6e'
geocoder = OpenCageGeocode(key)

# Cria a instância da classe
# arquivo = 'portalbio_export_17-10-2019-13-06-22.csv'
arquivo = 'portalbio_export_16-10-2019-14-39-54.csv'
preparador = Preparador(arquivo)

# Retorna o dataframe para checagem
mydf = preparador.get_df()
pprint(mydf)

# Printa a quantidade de linhas55
# A quantidade de linhas indica a quantidade de espécies registradas neste documento
print("Quantidade de espécies registradas: ", len(mydf))

     Nome da instituicao Sigla da instituicao  \
0        Sem Informações               ICMBio   
1        Sem Informações               ICMBio   
2        Sem Informações               ICMBio   
3        Sem Informações               ICMBio   
4        Sem Informações               ICMBio   
...                  ...                  ...   
3907     Sem Informações               ICMBio   
3908     Sem Informações               ICMBio   
3909     Sem Informações               ICMBio   
3910     Sem Informações               ICMBio   
3911     Sem Informações               ICMBio   

                                  Nome da base de dados  \
0                 Atlas de Registro de Aves Brasileiras   
1                 Atlas de Registro de Aves Brasileiras   
2                 Atlas de Registro de Aves Brasileiras   
3                 Atlas de Registro de Aves Brasileiras   
4                 Atlas de Registro de Aves Brasileiras   
...                                                 ...  

## Verificação dos dados faltantes no arquivo CSV

In [108]:
print("Dicionário com as informações sobre dados faltantes.")
print("No formato {campo: porcentagem de células sem dados}")
preparador.search_nan()

Dicionário com as informações sobre dados faltantes.
No formato {campo: porcentagem de células sem dados}


{'Nome da instituicao': 100.0,
 'Sigla da instituicao': 0.0,
 'Nome da base de dados': 0.0,
 'Sigla da base de dados': 0.0,
 'Responsavel pelo registro': 0.07668711656441718,
 'Numero do registro no portal': 0.0,
 'Numero do registro na base de dados': 6.007157464212679,
 'Data do registro': 3.502044989775051,
 'Data do evento': 54.4478527607362,
 'Data de Carencia': 0.0,
 'Nome cientifico': 1.687116564417178,
 'Nome comum': 74.20756646216769,
 'Nome cientifico na base de dados': 0.0,
 'Nivel taxonomico': 1.687116564417178,
 'Numero de individuos': 51.35480572597137,
 'Reino': 100.0,
 'Filo': 3.1697341513292434,
 'Classe': 3.7576687116564416,
 'Ordem': 4.294478527607362,
 'Familia': 4.729038854805726,
 'Genero': 16.51329243353783,
 'Especie': 37.44887525562372,
 'Estado de conservacao': 0.0,
 'Categoria de Ameaca': 85.12269938650307,
 'Localidade': 1.9427402862985685,
 'Pais': 3.425357873210634,
 'Estado/Provincia': 10.557259713701432,
 'Municipio': 42.12678936605317,
 'Status de Sensi

## Verificação do nível taxonômico de cada espécie registrada

In [109]:
print("Dicionário com as informações sobre dados faltantes.")
print("No formato {número da espécie: nível taxonômico}")
preparador.n_tax()

Dicionário com as informações sobre dados faltantes.
No formato {número da espécie: nível taxonômico}


{0: 'Especie',
 1: 'Especie',
 2: 'Especie',
 3: 'Especie',
 4: 'Especie',
 5: 'Especie',
 6: 'Especie',
 7: 'Especie',
 8: 'Especie',
 9: 'Especie',
 10: 'Especie',
 11: 'Especie',
 12: 'Especie',
 13: 'Especie',
 14: 'Especie',
 15: 'Especie',
 16: 'Especie',
 17: 'Especie',
 18: 'Especie',
 19: 'Especie',
 20: 'Especie',
 21: 'Especie',
 22: 'Especie',
 23: '',
 24: '',
 25: '',
 26: '',
 27: 'Especie',
 28: 'Especie',
 29: 'Especie',
 30: 'Especie',
 31: 'Especie',
 32: 'Especie',
 33: 'Especie',
 34: 'Genero',
 35: 'Especie',
 36: 'Genero',
 37: 'Genero',
 38: 'Genero',
 39: 'Genero',
 40: 'Genero',
 41: 'Genero',
 42: 'Genero',
 43: 'Especie',
 44: 'Genero',
 45: 'Familia',
 46: 'Especie',
 47: 'Familia',
 48: 'Genero',
 49: 'Genero',
 50: 'Genero',
 51: 'Especie',
 52: 'Especie',
 53: 'Especie',
 54: 'Genero',
 55: 'Especie',
 56: 'Especie',
 57: 'Genero',
 58: 'Genero',
 59: 'Familia',
 60: 'Especie',
 61: 'Genero',
 62: '',
 63: 'Genero',
 64: 'Genero',
 65: 'Especie',
 66: 'E

## Filtragem do dataframe

In [112]:
print("Gera um novo dataframe com a filtragem desejada.")
my_filter = preparador.filter()

Gera um novo dataframe com a filtragem desejada.
Filtrar dados por:
1. Estado
2. Nome de espécie
3. Categoria de ameaça
Digite o número da opção: 1
Digite a sigla do estado: PE
Ocorrências com o filtro aplicado: 1360
     Nome da instituicao Sigla da instituicao  \
0        Sem Informações               ICMBio   
1        Sem Informações               ICMBio   
2        Sem Informações               ICMBio   
3        Sem Informações               ICMBio   
4        Sem Informações               ICMBio   
...                  ...                  ...   
3865     Sem Informações               ICMBio   
3866     Sem Informações               ICMBio   
3867     Sem Informações               ICMBio   
3910     Sem Informações               ICMBio   
3911     Sem Informações               ICMBio   

                                  Nome da base de dados  \
0                 Atlas de Registro de Aves Brasileiras   
1                 Atlas de Registro de Aves Brasileiras   
2                

## Plot da localização das espécies

In [113]:
# O plot é de todas as ocorrências do filtro acima
mapa = preparador.plot_map(my_filter)
display(mapa)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …